# Preprocessing
\
The raw data are point clouds generated by MLS (mobile LiDAR). The very first preprocessing step involves processing by the SLAM algorithm (more specifically, via the application provided by GeoSLAM), and unzip files.

Then, the next part consists in classifying ground points, generating a DTM, and extracting vegetation points in a slice parallel to DTM. This step is carried out using the opensource software Computree, and is detailed in the file computree_steps.xsct2 (it is ready to use, only the input files need to be selected and the output folder specified, parameters can also be viewed and modified).

Note: depending on the device you are using, it may be necessary to tile the point cloud before processing with Computree. In this case, make sure you convert to point format 7 and merge the output tile files before moving on. I recommend using CloudCompare for tiling and for converting .laz to .las.

*Output folder: 'computree_outputs'*

# Clustering
\
*Input folder: 'computree_outputs'*

In [2]:
# Loading packages and modules
import os
import glob
from clustering import ClEngine, Cluster

In [5]:
las_files_path = 'computree_outputs' # location of preprocessed files
clustered_files_path = 'cluster_outputs' # location of clustered files

# Listing all .las files to cluster
las_files = glob.glob('computree_outputs/*.las')

# Listing clustered files
clustered = glob.glob('cluster_outputs/*.las')
clustered_names = [os.path.splitext(os.path.basename(file))[0] for file in clustered]

File 07_04_ct.las loaded successfully.
Clustering 07_04_ct.las points...


KeyboardInterrupt: 

In [ ]:
for file in las_files:
    
    # Checking if the file is not already clustered
    if os.path.splitext(os.path.basename(file))[0]+'_clusters' not in clustered_names:
        
        cl = ClEngine(file)
        
        # Cluster .las file points
        cl.DBSCAN_clustering(eps=0.05, min_samples=100) # distance parameters are in meters
        
        # Filter clusters given on a minimum number of points and minimum length
        cl.filtering(nb_points=500, min_dist=1)
        
        # Draw a (very) basic representation of the clusters
        cl.draw_clusters()
        
        # Save clustering results in new .las files (filename_clusters.las)
        cl.save_clusters()

# Cylinder fitting
\
A RANSAC cylinder fitting operation must be carried out. Here, it is carried out in CloudCompare, with the following parameters:
    -...

This step is also a visual validation step.

The point clouds of detected cylindrical shapes (not mesh !) must be exported in cloud ascii format (.txt), with headers as column titles. As there will be mutliple files, make sure you save them in a sub-folder.

*Output folder: 'shapes_raw/plot_name_or_reference_or_what_you_want'*

# Image classification

First, we need to create images from the previously generated points clouds of cylindrical shapes. The images are created from graphs in which the colour of the points depends on the z coordinate.

In [ ]:
# Loading modules
import image_creation as imgcreate
import image_classification as imgclf

In [ ]:
# Path of folder with folders containing the files from which to create images
path_raw = 'shapes_raw'

# Path of folder containing image folders
dataset_path = 'CNN_data'

# Size of images
image_size_inches = (4, 4) # in inches
image_size = (217, 217) # in pixels

In [ ]:
# Browse all files and create images from each file
for folder in glob.glob(path_raw+'/*'):
    
    folder_name = os.path.splitext(os.path.basename(folder))[0]
    dest = dataset_path + '/' + folder_name
    
    imgcreate.image_generator(data_folder=folder, img_folder=dest, figsize=image_size_inches)

Now that the images are created, we can train an image classifier.

In [ ]:
folder = 'CNN_data/07_04' # location of image data to train the model

train_ds, val_ds = imgclf.dataset_generation(folder)
train_ds_augmented = imgclf.augmentation(train_ds)

model = imgclf.make_model()